# `clean_lat_long()`: Clean and validate geographical coordinates

## Introduction

The function `clean_lat_long()` cleans and converts a column containing latitudinal and longitudinal coordinates into their desired format. The function `validate_lat_long()` will validate either a single coordinate or a column of coordinates, returning True if the value is valid, and False otherwise.

* Introduce supported latitudinal and longitudinal supported format:

  * Decimal degrees (dd): 41.5
  * Decimal degrees hemisphere (ddh): "41.5° N"
  * Degrees minutes (dm): "41° 30′ N"
  * Degrees minutes seconds (dms): "41° 30′ 0″ N"


* Handle invalid parsing with the `errors` parameter:

  * ‘coerce’ (default), invalid parsing will be set as NaN
  * ‘ignore’, then invalid parsing will return the input
  * ‘raise’, then invalid parsing will raise an exception
  
The **Report**. Details
* How many values were cleaned (transformed)
* How many values could not be parsed
* And the data summary: how many values in the correct format, and how many null values
  
The following sections demonstrate the functionality of `clean_lat_long()` and `validate_lat_long()`. 

### Dirty dataset

In [4]:
import pandas as pd
import numpy as np
df = pd.DataFrame({"messy_lat_long": 
                   [(41.5, -81.0), "41.5;-81.0", "41.5,-81.0", "41.5 -81.0", 
                    "41.5° N, 81.0° W", "-41.5 S;81.0 E", 
                    "23 26m 22s N 23 27m 30s E", "23 26' 22\" N 23 27' 30\" E", 
                    "UT: N 39°20' 0'' / W 74°35' 0''", "hello", np.nan, "NULL"]
                  })
df

,messy_lat_long
0,"(41.5, -81.0)"
1,41.5;-81.0
2,"41.5,-81.0"
3,41.5 -81.0
4,"41.5° N, 81.0° W"
5,-41.5 S;81.0 E
6,23 26m 22s N 23 27m 30s E
7,"23 26' 22"" N 23 27' 30"" E"
8,UT: N 39°20' 0'' / W 74°35' 0''
9,hello


## 1. Default `clean_lat_long()`

In [6]:
from dataprep.clean import clean_lat_long
clean_lat_long(df, "messy_lat_long")

Latitude and Longitude Cleaning Report:
	7 values cleaned (58.33%)
	2 values unable to be parsed (16.67%), set to NaN
Result contains 8 (66.67%) values in the correct format and 4 null values (33.33%)


,messy_lat_long,messy_lat_long_clean
0,"(41.5, -81.0)","(41.5, -81.0)"
1,41.5;-81.0,"(41.5, -81.0)"
2,"41.5,-81.0","(41.5, -81.0)"
3,41.5 -81.0,"(41.5, -81.0)"
4,"41.5° N, 81.0° W","(41.5, -81.0)"
5,-41.5 S;81.0 E,NaN
6,23 26m 22s N 23 27m 30s E,"(23.4394, 23.4583)"
7,"23 26' 22"" N 23 27' 30"" E","(23.4394, 23.4583)"
8,UT: N 39°20' 0'' / W 74°35' 0'',"(39.3333, -74.5833)"
9,hello,NaN


Note I consider (41.5, -81.0) not "cleaned" in the report since it's resulting format is the same as the input. Also, "-41.5 S;81.0 E" is invalid because if a coordinate has a hemisphere it cannot contain a negative decimal value.

## 2. Output formats:

### decimal degrees hemisphere

In [3]:
clean_lat_long(df, "messy_lat_long", output_format="ddh")

Latitude and Longitude Cleaning Report:
	8 values cleaned (61.54%)
	2 values unable to be parsed (15.38%), set to NaN
Result contains 9 (69.23%) values in the correct format and 4 null values (30.77%)


,messy_lat_long,messy_lat_long_clean
0,"(41.5, -81.0)","41.5° N, 81.0° W"
1,41.5;-81.0,"41.5° N, 81.0° W"
2,"41.5,-81.0","41.5° N, 81.0° W"
3,41.5 -81.0,"41.5° N, 81.0° W"
4,"41.5° N, 81.0° W","41.5° N, 81.0° W"
5,41.5 S;81.0 E,"41.5° S, 81.0° E"
6,-41.5 S;81.0 E,NaN
7,23 26m 22s N 23 27m 30s E,"23.4394° N, 23.4583° E"
8,"23 26' 22"" N 23 27' 30"" E","23.4394° N, 23.4583° E"
9,UT: N 39°20' 0'' / W 74°35' 0'',"39.3333° N, 74.5833° W"


 ### degrees minutes

In [4]:
clean_lat_long(df, "messy_lat_long", output_format="dm")

Latitude and Longitude Cleaning Report:
	9 values cleaned (69.23%)
	2 values unable to be parsed (15.38%), set to NaN
Result contains 9 (69.23%) values in the correct format and 4 null values (30.77%)


,messy_lat_long,messy_lat_long_clean
0,"(41.5, -81.0)","41° 30′ N, 81° 0′ W"
1,41.5;-81.0,"41° 30′ N, 81° 0′ W"
2,"41.5,-81.0","41° 30′ N, 81° 0′ W"
3,41.5 -81.0,"41° 30′ N, 81° 0′ W"
4,"41.5° N, 81.0° W","41° 30′ N, 81° 0′ W"
5,41.5 S;81.0 E,"41° 30′ S, 81° 0′ E"
6,-41.5 S;81.0 E,NaN
7,23 26m 22s N 23 27m 30s E,"23° 26.3667′ N, 23° 27.5′ E"
8,"23 26' 22"" N 23 27' 30"" E","23° 26.3667′ N, 23° 27.5′ E"
9,UT: N 39°20' 0'' / W 74°35' 0'',"39° 20′ N, 74° 35′ W"


### degrees minutes seconds

In [5]:
clean_lat_long(df, "messy_lat_long", output_format="dms")

Latitude and Longitude Cleaning Report:
	9 values cleaned (69.23%)
	2 values unable to be parsed (15.38%), set to NaN
Result contains 9 (69.23%) values in the correct format and 4 null values (30.77%)


,messy_lat_long,messy_lat_long_clean
0,"(41.5, -81.0)","41° 30′ 0″ N, 81° 0′ 0″ W"
1,41.5;-81.0,"41° 30′ 0″ N, 81° 0′ 0″ W"
2,"41.5,-81.0","41° 30′ 0″ N, 81° 0′ 0″ W"
3,41.5 -81.0,"41° 30′ 0″ N, 81° 0′ 0″ W"
4,"41.5° N, 81.0° W","41° 30′ 0″ N, 81° 0′ 0″ W"
5,41.5 S;81.0 E,"41° 30′ 0″ S, 81° 0′ 0″ E"
6,-41.5 S;81.0 E,NaN
7,23 26m 22s N 23 27m 30s E,"23° 26′ 22″ N, 23° 27′ 30″ E"
8,"23 26' 22"" N 23 27' 30"" E","23° 26′ 22″ N, 23° 27′ 30″ E"
9,UT: N 39°20' 0'' / W 74°35' 0'',"39° 20′ 0″ N, 74° 34′ 60″ W"


## 3. `split` parameter

TODO: introduce split parameter

In [6]:
clean_lat_long(df, "messy_lat_long", split=True)

Latitude and Longitude Cleaning Report:
	9 values cleaned (69.23%)
	2 values unable to be parsed (15.38%), set to NaN
Result contains 9 (69.23%) values in the correct format and 4 null values (30.77%)


,messy_lat_long,latitude,longitude
0,"(41.5, -81.0)",41.5000,-81.0000
1,41.5;-81.0,41.5000,-81.0000
2,"41.5,-81.0",41.5000,-81.0000
3,41.5 -81.0,41.5000,-81.0000
4,"41.5° N, 81.0° W",41.5000,-81.0000
5,41.5 S;81.0 E,-41.5000,81.0000
6,-41.5 S;81.0 E,NaN,NaN
7,23 26m 22s N 23 27m 30s E,23.4394,23.4583
8,"23 26' 22"" N 23 27' 30"" E",23.4394,23.4583
9,UT: N 39°20' 0'' / W 74°35' 0'',39.3333,-74.5833


In [7]:
clean_lat_long(df, "messy_lat_long", split=True, output_format="dm")

Latitude and Longitude Cleaning Report:
	9 values cleaned (69.23%)
	2 values unable to be parsed (15.38%), set to NaN
Result contains 9 (69.23%) values in the correct format and 4 null values (30.77%)


,messy_lat_long,latitude,longitude
0,"(41.5, -81.0)",41° 30′ N,81° 0′ W
1,41.5;-81.0,41° 30′ N,81° 0′ W
2,"41.5,-81.0",41° 30′ N,81° 0′ W
3,41.5 -81.0,41° 30′ N,81° 0′ W
4,"41.5° N, 81.0° W",41° 30′ N,81° 0′ W
5,41.5 S;81.0 E,41° 30′ S,81° 0′ E
6,-41.5 S;81.0 E,NaN,NaN
7,23 26m 22s N 23 27m 30s E,23° 26.3667′ N,23° 27.5′ E
8,"23 26' 22"" N 23 27' 30"" E",23° 26.3667′ N,23° 27.5′ E
9,UT: N 39°20' 0'' / W 74°35' 0'',39° 20′ N,74° 35′ W


## 4. `inplace` parameter
This just deletes the given column from the returned dataframe

In [8]:
clean_lat_long(df, "messy_lat_long", inplace=True)

Latitude and Longitude Cleaning Report:
	8 values cleaned (61.54%)
	2 values unable to be parsed (15.38%), set to NaN
Result contains 9 (69.23%) values in the correct format and 4 null values (30.77%)


,messy_lat_long_clean
0,"(41.5, -81.0)"
1,"(41.5, -81.0)"
2,"(41.5, -81.0)"
3,"(41.5, -81.0)"
4,"(41.5, -81.0)"
5,"(-41.5, 81.0)"
6,NaN
7,"(23.4394, 23.4583)"
8,"(23.4394, 23.4583)"
9,"(39.3333, -74.5833)"


Note the title change: `f"{original title}_clean"`

In [9]:
clean_lat_long(df, "messy_lat_long", split=True, inplace=True)

Latitude and Longitude Cleaning Report:
	9 values cleaned (69.23%)
	2 values unable to be parsed (15.38%), set to NaN
Result contains 9 (69.23%) values in the correct format and 4 null values (30.77%)


,latitude,longitude
0,41.5000,-81.0000
1,41.5000,-81.0000
2,41.5000,-81.0000
3,41.5000,-81.0000
4,41.5000,-81.0000
5,-41.5000,81.0000
6,NaN,NaN
7,23.4394,23.4583
8,23.4394,23.4583
9,39.3333,-74.5833


## 5. Latitude and longitude coordinates in separate columns

Clean a column with latitude coordinates

In [8]:
df = pd.DataFrame({"messy_lat": [" 30′ 0″ E", "41° 30′ N", "41 S", "80", "hello", "NA"]})
clean_lat_long(df, lat_col = "messy_lat")

Latitude and Longitude Cleaning Report:
	3 values cleaned (50.0%)
	2 values unable to be parsed (33.33%), set to NaN
Result contains 3 (50.0%) values in the correct format and 3 null values (50.0%)


,messy_lat,messy_lat_clean
0,30′ 0″ E,NaN
1,41° 30′ N,41.5
2,41 S,-41.0
3,80,80.0
4,hello,NaN
5,NA,NaN


In [19]:
df = pd.DataFrame({"messy_lat": ["30° E", "41° 30′ N", "41 S", "80", "hello", "NA"],
                      "messy_long": ["30° E", "41° 30′ N", "41 W", "80", "hello", "NA"]})
clean_lat_long(df, lat_col = "messy_lat", long_col = "messy_long")

Latitude and Longitude Cleaning Report:
	6 values cleaned (100.0%)
	4 values unable to be parsed (66.67%), set to NaN
Result contains 0 (0.0%) values in the correct format and 6 null values (100.0%)


/Users/brandonlockhart/dataprep/.venv/lib/python3.8/site-packages/dask/dataframe/core.py:4315: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))


,messy_lat,messy_long,latitude_longitude
0,30° E,30° E,"(nan, 30.0)"
1,41° 30′ N,41° 30′ N,"(41.5, nan)"
2,41 S,41 W,"(-41.0, -41.0)"
3,80,80,"(80.0, 80.0)"
4,hello,hello,"(nan, nan)"
5,NA,NA,"(nan, nan)"


In [20]:
clean_lat_long(df, lat_col = "messy_lat", long_col = "messy_long", split=True)

Latitude and Longitude Cleaning Report:
	6 values cleaned (100.0%)
	4 values unable to be parsed (66.67%), set to NaN
Result contains 0 (0.0%) values in the correct format and 6 null values (100.0%)


,messy_lat,messy_long,messy_lat_clean,messy_long_clean
0,30° E,30° E,NaN,30.0
1,41° 30′ N,41° 30′ N,41.5,NaN
2,41 S,41 W,-41.0,-41.0
3,80,80,80.0,80.0
4,hello,hello,NaN,NaN
5,NA,NA,NaN,NaN


## 6. `validate_lat_long()` 

TODO introduce `validate_lat_long()`. Same valid types as `clean_lat_long()`

In [22]:
from dataprep.clean import validate_lat_long
print(validate_lat_long("41° 30′ 0″ N"))
print(validate_lat_long("41.5 S;81.0 E"))
print(validate_lat_long("-41.5 S;81.0 E"))

False
True
False


In [23]:
df = pd.DataFrame({"messy_lat_long": 
                   [(41.5, -81.0), "41.5;-81.0", "41.5,-81.0", "41.5 -81.0", 
                    "41.5° N, 81.0° W", "-41.5 S;81.0 E", 
                    "23 26m 22s N 23 27m 30s E", "23 26' 22\" N 23 27' 30\" E", 
                    "UT: N 39°20' 0'' / W 74°35' 0''", "hello", np.nan, "NULL"]
                  })
validate_lat_long(df["messy_lat_long"])

0      True
1      True
2      True
3      True
4      True
5     False
6      True
7      True
8      True
9     False
10    False
11    False
Name: messy_lat_long, dtype: bool

### validate only one coordinate

In [24]:
validate_lat_long(df_lat["messy_lat"], lat_long = False, lat=True)

0    False
1     True
2     True
3     True
4    False
5    False
Name: messy_lat, dtype: bool